In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def generateRiblets(d, h, theta, l, d_frac):
    '''generates the vertices for a single riblet.  returns a list of 3D cartesian coords.
    d: riblet spacing
    h: riblet height
    theta: sharpness of riblet
    l: height of the system (distance to the lid driving the flow)
    d_frac: #the fraction of the separation used as the base for the trapezoid forming a side of the prism riblet'''
    w = h*math.tan(math.radians(theta/2)) #half of the base of the riblet triangle
    z = 0.1 #the length of the system in the z-direction.  Currently just used to make a dummy dimension for OpenFOAM's 2D quirks
    
    vertices = [None]*6 #list to hold all the vertices
    vertices[0] = (0, 0, 0) #origin
    vertices[1] = ((1.-d_frac)*d, 0, 0) #RHS edge of first cuboid
    vertices[2] = (d, 0, 0) #base of riblet LHS
    vertices[3] = (d+2.*w, 0, 0) #base of riblet RHS
    vertices[4] = ((1.+d_frac)*d + 2.*w, 0, 0) #LHS of edge cuboid
    vertices[5] = (2.*(d+w), 0, 0) #RHS of system base
    
    vertices = [np.array(v) for v in vertices]
    
    vertices_h = [None]*5 #the next row of vertices at the height of the riblet peak
    vertices_h[0] = (0, h, 0)
    vertices_h[1] = ((1.-d_frac)*d, h, 0)
    vertices_h[2] = (d+w, h, 0)
    vertices_h[3] = ((1.+d_frac)*d + 2.*w, h, 0)
    vertices_h[4] = (2.*(d+w), h, 0)
    
    vertices_h = [np.array(v) for v in vertices_h]
    vertices_l = [v + np.array([0, (l-h), 0]) for v in vertices_h] #the top row of vertices at the top edge of the system
    
    vertices.extend(vertices_h)
    vertices.extend(vertices_l)
    
#     for v in vertices:
#         plt.scatter(v[0], v[1])
#     plt.show()
    
    #vertices_other = vertices.copy() #duplicate of the vertices, but we add 0.1 to the z component as OpenFOAM requires 3D always
    vertices_other = [v + np.array([0., 0., z]) for v in vertices]
    
    vertices.extend(vertices_other)
    
    return vertices

In [4]:
def generateNRiblets(N, d, h, theta, l, d_frac):
    '''generates the vertices for a single riblet.  returns a list of 3D cartesian coords.
    d: riblet spacing
    h: riblet height
    theta: sharpness of riblet
    l: height of the system (distance to the lid driving the flow)'''
    w = h*math.tan(math.radians(theta/2)) #half of the base of the riblet triangle
    z = 0.1 #the length of the system in the z-direction.  Currently just used to make a dummy dimension for OpenFOAM's 2D quirks
    d_frac = 0.4 #the fraction of the separation used as the base for the trapezoid forming a side of the prism riblet
    
    initial_vertices = generateRiblets(d, h, theta, l, d_frac)
    all_vertices = initial_vertices.copy()
    system_size = initial_vertices[-1][0] #the total length of the system
    #print(system_size)
    for i in range(N-1):
        vertices_N = [v + np.array([system_size*(i+1), 0, 0]) for v in initial_vertices]
        all_vertices.extend(vertices_N)
        
    print(len(all_vertices))
    
        
    filtered_vertices = {array.tostring(): array for array in all_vertices}.values() #remove duplicate vertices
    print(len(filtered_vertices))

    for v in filtered_vertices:
        plt.scatter(v[0], v[1])
    plt.show()
    
    for i, elem in enumerate(filtered_vertices):
    #print("("+str(elem[0])+"\t"+str(elem[1])+"\t"+str(elem[2])+")")
        print("\t({0:5.3f}  {1:5.3f}  {2:5.3f})".format(elem[0], elem[1], elem[2]) + "\t //vertex "+str(i))
    

In [5]:
#vN = generateNRiblets(N=3, d=2., h=3., theta=45., l=10., d_frac=0.4)

In [6]:
def generateNRiblets_v2(N, d, h, theta, l, d_frac, plotRiblets=False):
    '''generates the vertices for multiple riblets.  returns a list of 3D cartesian coords.
    this version returns all the vertices on the bottom row, followed by the middle row, then the top row.
    then repeats for the other vertices in the z-direction.
    d: riblet spacing
    h: riblet height
    theta: sharpness of riblet
    l: height of the system (distance to the lid driving the flow)'''
    w = h*math.tan(math.radians(theta/2)) #half of the base of the riblet triangle
    z = 0.1 #the length of the system in the z-direction.  Currently just used to make a dummy dimension for OpenFOAM's 2D quirks
    d_frac = 0.4 #the fraction of the separation used as the base for the trapezoid forming a side of the prism riblet
    riblet_length = 2.*(d+w) #the total length of the riblet in the x-direction
    
    n_bottom = 6 #no. of vertices in the bottom row of a single riblet
    n_middle = 5 #no. of vertices in the middle row of a single riblet
    n_top = 5 #no. of vertices in top row of a single riblet
    
    bottom_vertices = [None]*(n_bottom*N) #make the lists slightly too big then remove the Nones later on
    middle_vertices = [None]*(n_middle*N) 
    
    #fill up the vertices repeating, then remove duplicates
    for i in range(N):
        #bottom vertices
        bottom_vertices[0+i*n_bottom] = (0 + i*riblet_length, 0, 0) #origin
        bottom_vertices[1+i*n_bottom] = ((1.-d_frac)*d + i*riblet_length, 0, 0) #RHS edge of first cuboid
        bottom_vertices[2+i*n_bottom] = (d + i*riblet_length, 0, 0) #base of riblet LHS
        bottom_vertices[3+i*n_bottom] = (d+2.*w + i*riblet_length, 0, 0) #base of riblet RHS
        bottom_vertices[4+i*n_bottom] = ((1.+d_frac)*d + 2.*w + i*riblet_length, 0, 0) #LHS of edge cuboid
        bottom_vertices[5+i*n_bottom] = (2.*(d+w) + i*riblet_length, 0, 0) #RHS of system base
        
        #middle vertices
        middle_vertices[0+i*n_middle] = (0 + i*riblet_length, h, 0)
        middle_vertices[1+i*n_middle] = ((1.-d_frac)*d + i*riblet_length, h, 0)
        middle_vertices[2+i*n_middle] = (d+w + i*riblet_length, h, 0)
        middle_vertices[3+i*n_middle] = ((1.+d_frac)*d + 2.*w + i*riblet_length, h, 0)
        middle_vertices[4+i*n_middle] = (2.*(d+w) + i*riblet_length, h, 0) 
        
    #print(bottom_vertices)
    bottom_vertices = [np.array(v) for v in bottom_vertices if v]    
    middle_vertices = [np.array(v) for v in middle_vertices if v]
    
    top_vertices = [v + np.array([0, (l-h), 0]) for v in middle_vertices] #the top row of vertices at the top edge of the system
    
    all_vertices = bottom_vertices
    all_vertices.extend(middle_vertices)
    all_vertices.extend(top_vertices)
    
    vertices_other = [v + np.array([0., 0., z]) for v in all_vertices] #add the other vertices in the z-direction
    all_vertices.extend(vertices_other)
    
    #print(len(all_vertices))
    filtered_vertices = list({array.tostring(): array for array in all_vertices}.values()) #remove duplicate vertices
    #print(len(filtered_vertices))
    
    if plotRiblets:
        plt.figure(figsize=(20,10))
        for v in filtered_vertices:
            plt.scatter(v[0], v[1])
        plt.show()

    #print(filtered_vertices[6*N-(N-1)-1])
    for i, elem in enumerate(filtered_vertices):
    #print("("+str(elem[0])+"\t"+str(elem[1])+"\t"+str(elem[2])+")")
        print("\t({0:5.3f}  {1:5.3f}  {2:5.3f})".format(elem[0], elem[1], elem[2]) + "\t //vertex "+str(i))

In [7]:
#vN = generateNRiblets_v2(N=3, d=2., h=3., theta=45., l=10., d_frac=0.4)

In [8]:
def generateHexBlocks(N):
    '''method to generate hex blocks for N prism riblets for use in openfoam.
    N = no. of riblets'''
    
    #the strings below are used to have their values read in from a file in the blockMeshDict folder
    ribletX_nC = '$ribletX_nC' #the no. of x-cells in the blocks on/above the riblets
    plateauX_nC = '$plateauX_nC' #the no. of x-cells in the blocks on/above the space between riblets
    surfaceY_nC = '$surfaceY_nC' #the no. of y-cells in the blocks on the surface
    fluidY_nC = '$fluidY_nC' #the n0. of y-cells in the blocks in the surrounding fluid (e.g. ocean)
    
    nCells_z = '1' #only one cell in z-direction
    
    n_blocks = 8 #this is the number of blocks in each riblet subsystem.  Just used for keeping track of the indexing
    
    nb = 6*N - (N-1) #the no. of vertices on the bottom row. -1 for the index of the last bottom vertex. nb is the index of the first mid vertex
    nm = 2*nb - N #the no of vertices in the middle and bottom rows. -1 for the index of the last middle vertex. nm is the index for the first top vertex
    nt = 3*nb - 2*N #the total number of vertices in the system. -1 for the very last index.  The other z-slice starts at index nt
    z0 = nt #the index of the first vertex in the other z slice
    
    hex_list = [] #this is the list that will hold all the hex blocks
    
    for i in range(N):
        bot_offset = 5*i #this moves everything along properly when we move to the next riblet
        mid_offset = 4*i
        top_offset = 4*i
        
        #the lower row of blocks, along the riblet surface
        block0 = [0+bot_offset, 1+bot_offset, nb+1+mid_offset, nb+0+mid_offset, 0+bot_offset+z0, 1+bot_offset+z0, nb+1+mid_offset+z0, nb+0+mid_offset+z0]
        block1 = [1+bot_offset, 2+bot_offset, nb+2+mid_offset, nb+1+mid_offset, 1+bot_offset+z0, 2+bot_offset+z0, nb+2+mid_offset+z0, nb+1+mid_offset+z0]
        block2 = [3+bot_offset, 4+bot_offset, nb+3+mid_offset, nb+2+mid_offset, 3+bot_offset+z0, 4+bot_offset+z0, nb+3+mid_offset+z0, nb+2+mid_offset+z0]
        block3 = [4+bot_offset, 5+bot_offset, nb+4+mid_offset, nb+3+mid_offset, 4+bot_offset+z0, 5+bot_offset+z0, nb+4+mid_offset+z0, nb+3+mid_offset+z0]
        
        #the upper row of blocks, bulk fluid environment
        block4 = [nb+0+mid_offset, nb+1+mid_offset, nm+1+top_offset, nm+0+top_offset, nb+0+mid_offset+z0, nb+1+mid_offset+z0, nm+1+top_offset+z0, nm+0+top_offset+z0]
        block5 = [nb+1+mid_offset, nb+2+mid_offset, nm+2+top_offset, nm+1+top_offset, nb+1+mid_offset+z0, nb+2+mid_offset+z0, nm+2+top_offset+z0, nm+1+top_offset+z0]
        block6 = [nb+2+mid_offset, nb+3+mid_offset, nm+3+top_offset, nm+2+top_offset, nb+2+mid_offset+z0, nb+3+mid_offset+z0, nm+3+top_offset+z0, nm+2+top_offset+z0]
        block7 = [nb+3+mid_offset, nb+4+mid_offset, nm+4+top_offset, nm+3+top_offset, nb+3+mid_offset+z0, nb+4+mid_offset+z0, nm+4+top_offset+z0, nm+3+top_offset+z0]
        
        hex_list.append(block0)
        hex_list.append(block1)
        hex_list.append(block2)
        hex_list.append(block3)
        hex_list.append(block4)
        hex_list.append(block5)
        hex_list.append(block6)
        hex_list.append(block7)
        
    
    #now we print the blocks in the correct format so they can be pasted into the blockMeshDict file.
    block_i = 0
    riblet_i = 0
    for i, elem in enumerate(hex_list):
        
        block_i = i%(n_blocks)
        
        
        elem = [str(v) for v in elem]
        vertex_string = " ".join(elem) #the list of the vertices in the block
        
        nCells_string = ""
        nCells_x = ""
        nCells_y = ""
        
        #print(i)
        #print(block_i%(n_blocks//2))
        
        if block_i%(n_blocks//2) == 0 or block_i%(n_blocks//2) == 3:
            nCells_x = plateauX_nC
        else:
            nCells_x = ribletX_nC
            
        
        if i%n_blocks < 4:
            nCells_y = surfaceY_nC
        else:
            nCells_y = fluidY_nC
            
        
        
        hex_string = "hex ("+vertex_string+") \t("+nCells_x+" "+nCells_y+" "+nCells_z+") \tsimpleGrading (1 1 1) "+("\t//riblet "+str(riblet_i)+", block "+str(block_i)).rjust(22, " ")
        print(hex_string)
        if i%n_blocks == n_blocks-1 and i > 0:
            riblet_i += 1
            print()

        

In [9]:
generateHexBlocks(2)

hex (0 1 12 11 29 30 41 40) 	($plateauX_nC $surfaceY_nC 1) 	simpleGrading (1 1 1)   	//riblet 0, block 0
hex (1 2 13 12 30 31 42 41) 	($ribletX_nC $surfaceY_nC 1) 	simpleGrading (1 1 1)   	//riblet 0, block 1
hex (3 4 14 13 32 33 43 42) 	($ribletX_nC $surfaceY_nC 1) 	simpleGrading (1 1 1)   	//riblet 0, block 2
hex (4 5 15 14 33 34 44 43) 	($plateauX_nC $surfaceY_nC 1) 	simpleGrading (1 1 1)   	//riblet 0, block 3
hex (11 12 21 20 40 41 50 49) 	($plateauX_nC $fluidY_nC 1) 	simpleGrading (1 1 1)   	//riblet 0, block 4
hex (12 13 22 21 41 42 51 50) 	($ribletX_nC $fluidY_nC 1) 	simpleGrading (1 1 1)   	//riblet 0, block 5
hex (13 14 23 22 42 43 52 51) 	($ribletX_nC $fluidY_nC 1) 	simpleGrading (1 1 1)   	//riblet 0, block 6
hex (14 15 24 23 43 44 53 52) 	($plateauX_nC $fluidY_nC 1) 	simpleGrading (1 1 1)   	//riblet 0, block 7

hex (5 6 16 15 34 35 45 44) 	($plateauX_nC $surfaceY_nC 1) 	simpleGrading (1 1 1)   	//riblet 1, block 0
hex (6 7 17 16 35 36 46 45) 	($ribletX_nC $surfaceY_nC 1) 

In [10]:
def generateFloorFaces(N):
    '''generates the faces for the bottom floor of the system.
    these are defined anticlockwise to the face normal vector pointing out of the system
    N: no. of riblets'''
    
    nb = 6*N - (N-1) #the no. of vertices on the bottom row. -1 for the index of the last bottom vertex. nb is the index of the first mid vertex
    nm = 2*nb - N #the no of vertices in the middle and bottom rows. -1 for the index of the last middle vertex. nm is the index for the first top vertex
    nt = 3*nb - 2*N #the total number of vertices in the system. -1 for the very last index.  The other z-slice starts at index nt
    z0 = nt #the index of the first vertex in the other z slice
    
    face_list = []
    
    for i in range(N):
        bot_offset = 5*i #this moves everything along properly when we move to the next riblet
        mid_offset = 4*i
        top_offset = 4*i
        
        face0 = (0+bot_offset, z0+0+bot_offset, z0+1+bot_offset, 1+bot_offset)
        face1 = (1+bot_offset, z0+1+bot_offset, z0+2+bot_offset, 2+bot_offset)
        face2 = (2+bot_offset, z0+2+bot_offset, z0+2+nb+mid_offset, 2+nb+mid_offset)
        face3 = (2+nb+mid_offset, z0+2+nb+mid_offset, z0+3+bot_offset, 3+bot_offset)
        face4 = (3+bot_offset, z0+3+bot_offset, z0+4+bot_offset, 4+bot_offset)
        face5 = (4+bot_offset, z0+4+bot_offset, z0+5+bot_offset, 5+bot_offset)
        
        face_list.append(face0)
        face_list.append(face1)
        face_list.append(face2)
        face_list.append(face3)
        face_list.append(face4)
        face_list.append(face5)
        
    print('lower wall:\n')
    for i in face_list:
        i = [str(f) for f in i]
        print('('+' '.join(i)+')')
        
    print()

In [11]:
#generateFloorFaces(1)

In [12]:
def generateTopFaces(N):
    '''generates the faces for the top wall of the system.
    these are defined anticlockwise wrt the normal face vector pointing out of the system.
    N: no. of riblets'''
    
    nb = 6*N - (N-1) #the no. of vertices on the bottom row. -1 for the index of the last bottom vertex. nb is the index of the first mid vertex
    nm = 2*nb - N #the no of vertices in the middle and bottom rows. -1 for the index of the last middle vertex. nm is the index for the first top vertex
    nt = 3*nb - 2*N #the total number of vertices in the system. -1 for the very last index.  The other z-slice starts at index nt
    z0 = nt #the index of the first vertex in the other z slice
    
    face_list = []
    
    for i in range(nt-nm-1):
        face = (nm+i, nm+i+1, z0+nm+i+1, z0+nm+i)
        face_list.append(face)
        
    print('upper wall:\n')
    for i in face_list:
        i = [str(f) for f in i]
        print('('+' '.join(i)+')')
        
    print()

In [13]:
#generateTopFaces(2)

In [14]:
def generateSideFaces(N):
    '''generates the faces for the LHS and RHS walls in the system.
    these are defined anticlockwise wrt the normal face vector pointing out of the system.
    N: no. of riblets
    '''
    
    nb = 6*N - (N-1) #the no. of vertices on the bottom row. -1 for the index of the last bottom vertex. nb is the index of the first mid vertex
    nm = 2*nb - N #the no of vertices in the middle and bottom rows. -1 for the index of the last middle vertex. nm is the index for the first top vertex
    nt = 3*nb - 2*N #the total number of vertices in the system. -1 for the very last index.  The other z-slice starts at index nt
    z0 = nt #the index of the first vertex in the other z slice
    
    face_list_LHS = []
    face_list_RHS = []
    
    face0_LHS = (0, nb, nb+z0, z0)
    face1_LHS = (nb, nm, nm+z0, nb+z0)
    face_list_LHS.append(face0_LHS)
    face_list_LHS.append(face1_LHS)
    
    face0_RHS = (nb-1, z0+nb-1, z0+nm-1, nm-1)
    face1_RHS = (nm-1, z0+nm-1, z0+nt-1, nt-1)
    face_list_RHS.append(face0_RHS)
    face_list_RHS.append(face1_RHS)
    
    print('left wall:\n')
    for i in face_list_LHS:
        i = [str(f) for f in i]
        print('('+' '.join(i)+')')
        
    print('\nright wall:\n')
    for i in face_list_RHS:
        i = [str(f) for f in i]
        print('('+' '.join(i)+')')
    
    print()

In [15]:
#generateSideFaces(2)

In [16]:
def generateAllFaces(N):
    
    generateSideFaces(N)
    generateFloorFaces(N)
    generateTopFaces(N)

In [18]:
#N riblets in micron scale with 0.05 spacing, 0.1 height and base width
N = 1
d = 92.
h = 42.
#base_width = 0.05
#theta=math.degrees(math.atan(base_width/h))
theta=42
#print(theta)
l = 200.
generateNRiblets_v2(N=N, d=d, h=h, theta=theta, l=l, d_frac=0.4)
generateHexBlocks(N)
generateAllFaces(N)

	(0.000  0.000  0.000)	 //vertex 0
	(55.200  0.000  0.000)	 //vertex 1
	(92.000  0.000  0.000)	 //vertex 2
	(124.245  0.000  0.000)	 //vertex 3
	(161.045  0.000  0.000)	 //vertex 4
	(216.245  0.000  0.000)	 //vertex 5
	(0.000  42.000  0.000)	 //vertex 6
	(55.200  42.000  0.000)	 //vertex 7
	(108.122  42.000  0.000)	 //vertex 8
	(161.045  42.000  0.000)	 //vertex 9
	(216.245  42.000  0.000)	 //vertex 10
	(0.000  200.000  0.000)	 //vertex 11
	(55.200  200.000  0.000)	 //vertex 12
	(108.122  200.000  0.000)	 //vertex 13
	(161.045  200.000  0.000)	 //vertex 14
	(216.245  200.000  0.000)	 //vertex 15
	(0.000  0.000  0.100)	 //vertex 16
	(55.200  0.000  0.100)	 //vertex 17
	(92.000  0.000  0.100)	 //vertex 18
	(124.245  0.000  0.100)	 //vertex 19
	(161.045  0.000  0.100)	 //vertex 20
	(216.245  0.000  0.100)	 //vertex 21
	(0.000  42.000  0.100)	 //vertex 22
	(55.200  42.000  0.100)	 //vertex 23
	(108.122  42.000  0.100)	 //vertex 24
	(161.045  42.000  0.100)	 //vertex 25
	(216.245  42.000  0